In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [3]:

llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
# create a state type

class SimpleLLMState(TypedDict):
    
    input_text: str
    output_text: str

In [10]:
def llm_qa(state: SimpleLLMState) -> SimpleLLMState:
    """
    A simple LLM function that takes input text and returns output text.
    """
    # Extract input text from the state 
    input_text = state['input_text']

    # Form the prompt for the model
    prompt = f"Answer the following question: {input_text}"

    # ask the model to invoke

    response = llm.invoke(prompt)

    # Update the state with the model's response

    state['output_text'] = response.content
    
    return state

In [11]:
# create graph

graph = StateGraph(SimpleLLMState)

graph.add_node("input_text", llm_qa)

graph.add_edge(START, "input_text")
graph.add_edge("input_text", END)

# compile the graph
workflow = graph.compile()

In [13]:
# execute the graph

initial_state = {'input_text': 'What is the capital of France?'}
final_state = workflow.invoke(initial_state)

print(final_state['output_text'])  

The capital of France is Paris.
